# Advent of Code 2022: Day 9
https://adventofcode.com/2022/day/9


## Part 1
Find the number of spaces the tail touched at least once

### Get the data into a list of strings
One observation: 'U 1'

In [1]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
# Remove empty value at the bottom of the list
data_list = data_list[:-1]

### Transform to a list of lists
One observation: ['U', 1]

In [2]:
data_lists = []
for item in data_list:
  # Split the item on empty space.
  temp_list = item.split(' ')

  # Transform the number part to an integer.
  temp_list[1] = int(temp_list[1])

  # Append to the final list.
  data_lists.append(temp_list)

### Set up helper functions

#### Helper function to create the gameboard

In [3]:
def setup_gameboard(move_list):
  
  # Set up dictionary of moves.
  space = {"U" : 0,
           "R" : 0,
           "L" : 0,
           "D" : 0}
  
  # Fill the dictionary with the
  # total number of moves in
  # every direction.
  for move in move_list:
    space[move[0]] += move[1]

  # Find the direction with the 
  # largest number of moves,
  # add 30 extra for safety.   
  maxdirr = max(space.items())[1] + 30

  # Set up a 2d array of zeros,
  # with rows and colums equal
  # to the calculated maxdirr.
  board = [[0 for x in range(maxdirr)] for y in range(maxdirr)]
  return board

#### Helper function to execute moves

In [4]:
def execute_move(move, Hpos, Tpos, gameboard, tail=True):
  newTpos = Tpos.copy()

  #############################
  # Move the head based on    #
  # the given direction       #
  #############################
  if move[0]=="U":            #
    Hpos[0] -= 1              #
  if move[0]=="D":            #
    Hpos[0] += 1              #
  if move[0]=="R":            #
    Hpos[1] += 1              #
  if move[0]=="L":            #
    Hpos[1] -= 1              #
  if move[0] == "UR":         #
    Hpos[0] -=1               #
    Hpos[1] +=1               #
  if move[0] == "UL":         #
    Hpos[0] -=1               #
    Hpos[1] -=1               #
  if move[0] == "DR":         #
    Hpos[0] +=1               #
    Hpos[1] +=1               #
  if move[0] == "DL":         #
    Hpos[0] +=1               #
    Hpos[1] -=1               #
###############################

  #################################
  # If the rows are the name      #
  #################################
  if (Tpos[0] == Hpos[0]):        #
    if((Tpos[1]-Hpos[1])>1):      #
      # Move one left             #
      newTpos[1] -=1              #
      # Move one right            #
    elif((Tpos[1]-Hpos[1]<-1)):   #
      newTpos[1] +=1              #
  #################################

  #################################
  # If the columns are the same   #
  #################################  
  elif (Tpos[1] == Hpos[1]):      #
    if((Tpos[0]-Hpos[0])>1):      #
      # Move one up               #
      newTpos[0] -=1              #
    elif((Tpos[0]-Hpos[0]<-1)):   #
      # Move one down             #
      newTpos[0] +=1              #
   ################################
  else:

    # If the tail is 2 rows below the head
    if(Tpos[0]-Hpos[0] > 1):

      # If the tail is 1 or more columns
      # to the right of the head
      if(Tpos[1]-Hpos[1] >= 1):

        # Move 1 up and 1 left
        newTpos[1] -=1
        newTpos[0] -=1
      
      # If the tail is 1 or more columns
      # to the left of the head
      elif(Tpos[1]-Hpos[1] <= -1):

        # Move 1 up and 1 right
        newTpos[1] +=1
        newTpos[0] -=1

    # If the tail is 2 rows above the head
    elif(Tpos[0]-Hpos[0] < -1):

      # If the tail is 1 or more columns
      # to the right of the head
      if(Tpos[1]-Hpos[1] >= 1):

        # Move 1 down and 1 left
        newTpos[1] -=1
        newTpos[0] +=1

      # If the tail is 1 or more columns
      # to the left of the head
      elif(Tpos[1]-Hpos[1] <= -1):

        # Move 1 down and 1 right
        newTpos[1] +=1
        newTpos[0] +=1

    # If the tail is 2 columns to the right of the head
    elif(Tpos[1]-Hpos[1] > 1):

      # If the tail is 1 row below the head
      if(Tpos[0]-Hpos[0] == 1):

        # Move 1 left and 1 up
        newTpos[1] -=1
        newTpos[0] -=1

      # If the tail is 1 row above the head 
      elif(Tpos[0]-Hpos[0] == -1):

        # Move 1 left and 1 down
        newTpos[1] -=1
        newTpos[0] +=1

    # If the tail is 2 columns to the left of the head
    elif(Tpos[1]-Hpos[1] < -1):

      # If the tail is 1 row below the head
      if(Tpos[0]-Hpos[0] == 1):

        # Move 1 right and 1 up
        newTpos[1] +=1
        newTpos[0] -=1

      # If the tail is 1 row above the head  
      elif(Tpos[0]-Hpos[0] == -1):

        # Move 1 right and 1 down
        newTpos[1] +=1
        newTpos[0] +=1


  # If tail is true, the tail is the actual
  # tail of the entire rope. If that is the 
  # case, update the gameboard if the tail
  # is in a position it has not been at before.
  if tail == True:
    if gameboard[newTpos[0]][newTpos[1]] == 0:
      gameboard[newTpos[0]][newTpos[1]] = 1
  
  return Hpos, newTpos, gameboard

#### Helper function to calculate the final sum

In [5]:
def sum_visited_spaces(gameboard):
  res = 0
  for i in gameboard:
    for j in i:
      res += j
  return res

#### Helper function to find which direction to move in

In [6]:
def find_direction(pos1, pos2):
  # If the rows are the same
  if(pos1[0] == pos2[0]):
    
    # If pos1 is to the 
    # right of pos2 
    if(pos1[1]-pos2[1] == 1):
      dirr = "R"
    
    # If pos1 is to the 
    # left of pos2
    elif(pos1[1]-pos2[1]==-1):
      dirr = "L"
  
  # If the columns are the same
  elif(pos1[1]== pos2[1]):

    # If pos1 is below pos2
    if(pos1[0]-pos2[0]==1):
      dirr = "D"
    
    # If pos1 is above pos2 
    elif(pos1[0]-pos2[0]==-1):
      dirr = "U"
  
  # If pos1 is below pos2
  elif(pos1[0]-pos2[0]==1):

    # If pos1 is to the 
    # right of pos2
    if(pos1[1]-pos2[1]==1):
      dirr= "DR"

    # If pos1 is to the 
    # left of pos2  
    elif(pos1[1]-pos2[1]==-1):
      dirr= "DL"

  # If pos1 is above pos2   
  elif(pos1[0]-pos2[0]==-1):

    # If pos1 is to the 
    # right of pos2
    if(pos1[1]-pos2[1]==1):
      dirr = "UR"

    # If pos1 is to the 
    # left of pos2   
    elif(pos1[1]-pos2[1]==-1):
      dirr = "UL"
  return dirr

### Function to play the game

In [7]:
def play_game(movelist, n_knots = 2):
  
  # Set up the game board and
  # the start position
  board = setup_gameboard(movelist)
  startpos = int(len(board)/2)

  # If the number of knots is
  # equal to 2, there is just a
  # head followed by a tail
  if n_knots == 2:
    
    # Set up initial positions
    # of the head and tail
    Hpos = [startpos, startpos]
    Tpos = [startpos, startpos]

    # Set the initial position
    # on the board to 1
    board[startpos][startpos] = 1

    # Go through the movelist
    for move in movelist:
      
      # Go through the number of
      # times a move should be executed
      for i in range(move[1]):
        Hpos, Tpos, board = execute_move(move, Hpos, Tpos, board)
  
  # If the number of knots
  # is greater than 2 more
  # steps are required
  else:
    
    # Set up the list of knot-positions
    knots = [[startpos, startpos]]
    for i in range(1,n_knots):
      knots.append([startpos, startpos])

    # Go through the movelist
    for move in movelist:

      # Go through the number of
      # times a move should be executed
      for i in range(move[1]):

        # Get positions of the
        # head and first knot
        Hpos = knots[0]
        Tpos1 = knots[1]

        # Keep track of if the current tail
        # is the true tail of the full rope
        tail = False

        # Move the head, while for the first
        # knot store the "potential" move in a 
        # temporary value
        Hpos, Tpos_temp, board = execute_move(move, Hpos, Tpos1, board, tail)

        # Update the knots list with
        # the new position of the head
        knots[0] = Hpos
        
        # If the first knots new position, after a 
        # potential move, is equal to its previous
        # position, it does not need to move. If so
        # continue to the next iteration of the loop. 
        if ((Tpos_temp[0] == Tpos1[0]) and (Tpos_temp[1]==Tpos1[1])):
          continue
        else:

          # If the first knot needs to move, start
          # at position 1 and go through until
          # the (n-2)nd position. 
          for i in range(1, len(knots)-1):
            
            # Using the position after potential
            # move and old position, find out
            # which direction this potential
            # move would have been in.
            dirr = find_direction(Tpos_temp, Tpos1)

            # Set the "inner move" to
            # this obtained direction
            innermove = [dirr,1]

            # Get the "inner tail" as
            # the (i+1)th position
            innerTpos = knots[i+1]
            
            # If (i+1) is equal to (n-2)
            # the "inner tail" is the 
            # tail of the full rope
            if (i+1) == (len(knots)-1):
              
              # In this case the
              # tail is the actual tail
              tail = True

              # Move the inner head
              # and tail
              Hpos, Tpos, board = execute_move(innermove, Tpos1, innerTpos, board, tail)

              # Update the positions of
              # the inner head and tail
              knots[i] = Hpos
              knots[i+1] = Tpos
              break

            # If (i+1) is not equal to (n-2)
            # the inner tail is not the actual tail.
            # If so, execute a move, where the
            # previous tail is now the head, and
            # the inner tail is the tail. Store
            # the potential move of the tail as 
            # was done previously.
            Hpos, Tpos_temp, board = execute_move(innermove, Tpos1, innerTpos, board, tail)

            # Update the position
            # of the current knot
            knots[i] = Hpos
            
            # As before, if the new position, after a 
            # potential move, is equal to the previous
            # position, the tail does not need to move.
            # If so break the inner loop.
            if Tpos_temp[0] == innerTpos[0] and Tpos_temp[1] == innerTpos[1]:
              break
            
            # Otherwise, set the inner tail
            # as the new inner head
            else:
              Tpos1 = innerTpos

  # Calculate the sum of all
  # visited spaces
  res = sum_visited_spaces(board)
  return res

In [8]:
res = play_game(data_lists)
res

6087

## Part 2


In [9]:
res2 = play_game(data_lists, n_knots= 10)
res2

2493